# Constructing 10-10 coordinates on segmented MRI scans

In [ ]:
import cedalion
import cedalion.io
import cedalion.geometry.segmentation
import cedalion.geometry.landmarks
from cedalion.imagereco.forward_model import TwoSurfaceHeadModel
import cedalion.datasets
import os.path
import pyvista

#pyvista.set_jupyter_backend("html")
pyvista.set_jupyter_backend("static")

## Load segmentation masks

This example constructs the 10-10 system on the Colin27 average brain.

In [ ]:
SEG_DATADIR, mask_files, landmarks_file = cedalion.datasets.get_colin27_segmentation()
masks, t_ijk2ras = cedalion.io.read_segmentation_masks(SEG_DATADIR, mask_files)

## Wrap the segmented head with derived surfaces in a TwoSurfaceHeadModel

In [ ]:
head = TwoSurfaceHeadModel.from_surfaces(
    segmentation_dir=SEG_DATADIR,
    mask_files = mask_files,
    brain_surface_file= os.path.join(SEG_DATADIR, "mask_brain.obj"),
    scalp_surface_file= os.path.join(SEG_DATADIR, "mask_scalp.obj"),
    landmarks_ras_file=landmarks_file,
    brain_face_count=None,
    scalp_face_count=None
)

## Transform the scalp surface from voxel space ('ijk') to RAS space ('aligned')

In [ ]:
scalp_surface = head.scalp
display(scalp_surface)
scalp_surface = scalp_surface.apply_transform(t_ijk2ras)
display(scalp_surface)

## Transform initial landmarks from voxel space ('ijk') to RAS space ('aligned')

In [ ]:
landmarks_ras = head.landmarks.points.apply_transform(t_ijk2ras)

## Construct landmarks

In [ ]:
lmbuilder = cedalion.geometry.landmarks.LandmarksBuilder1010(scalp_surface, landmarks_ras)
all_landmarks = lmbuilder.build()

## Visualize

In [ ]:
lmbuilder.plot()
display(all_landmarks)